Перед нами стоит задача предсказания числового значения на основе заданного текста. По заголовку мы должны предсказать его числовой рейтинг от 0.0 до 10.0 с точностью 0.1. 

В качестве библиотеки, реализующей модель русского языка, мы используем spaCy (в первую очередь из соображений скорости).

- Лемматизация
- Составление триграмм
- Категоризация отрезков текста

In [1]:
import pickle
import numpy as np
import pandas as pd

#import torch
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# https://spacy.io/models/ru
import spacy
spacy.prefer_gpu()
nlp = spacy.load("ru_core_news_sm")

from tqdm.auto import tqdm
tqdm.pandas()

DATASETS_PATH = '/home/leo/DATASETS'

with open('../sources.pickle', 'rb') as f:
    sources = pickle.load(f)
    
dfs = dict()
for source in sources:
    dfs[source] = pd.read_csv(f"{DATASETS_PATH}/{source}.csv", index_col=0, parse_dates=['post_time', 'parse_time'])
    dfs[source]['source'] = source

df = pd.concat(dfs[key] for key in dfs)
    
# удаляем дубликаты
df = df.drop_duplicates()    

# удаляем закрытые записи без просмотров
#df = df.drop(df[df.views_num == np.NINF].index)

df = pd.concat([dfs[key] for key in dfs])
df.views_num = df.views_num.apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))
df.views_num = np.log(df.views_num)/np.log(df.views_num.max())
df = df.drop(df[df.views_num == np.NINF].index)

/home/leo/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Оценка заголовка
Есть две модели: модель генерации заголовка и модель оценки заголовка. Начнем с модели оценки.

In [3]:
Xy = df[['title', 'views_num']]

# генерация дополнительных признаков
Xy.loc[:, ['title']] = Xy.title.apply(str)

Xy.loc[:, ['tokens']] = Xy.title.progress_apply(nlp)

# длина заголовка в символах
Xy.loc[:, ['len']] = Xy.title.apply(len)

# количество токенов
# ...

y = Xy.views_num
# X = Xy.drop(columns=['views_num'])
# Xy.sort_values(by='views_num', ascending=False)

  0%|          | 0/240582 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
with open(f"{DATASETS_PATH}/title.pickle", 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(Xy, f, pickle.HIGHEST_PROTOCOL)

Для проверки модели используем [кросс-валидацию](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9A%D1%80%D0%BE%D1%81%D1%81-%D0%B2%D0%B0%D0%BB%D0%B8%D0%B4%D0%B0%D1%86%D0%B8%D1%8F) по 5 фолдам.

In [ ]:
import 


In [ ]:
cloudpickle.dumps(nlp.tagger)

In [ ]:
x = 'Подборка бизнес-литературы для предпринимателей'

In [ ]:
DATASETS_PATH

In [ ]:
# def load_text(source, filename):
#     try:
#         with open(f'{DATASETS_PATH}/{source}/{filename}', 'r') as f:
#             return f.read()
#     except FileNotFoundError:
#         return ''
    
# df['full_text'] = df.apply(lambda x: load_text(x.source, x.filename), axis=1)